<h1 id="Road Safety Data Analysis"><span style="color: #ff0000;"> Accident Data Analysis (Dept. for Transport - United Kingdom) 2005-2014</span></h1>

<h2 style="text-align: left;">Initialize libraries, get data sets, consolidate and cleanse.</h2>

In [1]:
# Importing libraries and the data set
from pandas import Series, DataFrame
import pandas as pd
import glob
import numpy as np
import os
import matplotlib.pylab as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = 12, 4  # that's default image size for this interactive session
import scipy
from scipy import stats
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix,  precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import Binarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
from sklearn.metrics import roc_curve, auc

## Perform Logistic Regression using StatsModels

In [3]:
#Reading the data from 2005 - 2014 Accident Data
df = pd.read_csv("Accidents2005to2014.csv")
df['ACC_FATAL'] = np.where(df.Accident_Severity == 1, 1 , 0)

#df.head()

#drop unnecesary columns
df.drop(['Unnamed: 0', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude'], axis=1, inplace=True)
df.drop(['Date', 'Time', 'LSOA_of_Accident_Location'], axis=1, inplace=True)

df.columns = ['﻿Accident_Index','Accident_Severity','Number_of_Vehicles','Number_of_Casualties','Day_of_Week',
              'Speed_limit','Light_Conditions','Weather_Conditions','Urban_or_Rural_Area','ACC_FATAL']

# making speed limit a dummy column
dummy_ranks = pd.get_dummies(df['Speed_limit'], prefix='Speed_limit')

# create a clean data frame for the regression
cols_to_keep = ['Day_of_Week', 'Speed_limit', 'Light_Conditions', 
                'Weather_Conditions', 'Urban_or_Rural_Area','Number_of_Vehicles', 'Number_of_Casualties','ACC_FATAL']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'Speed_limit_2':])

# manually add the intercept to gaurantee that the residuals have a mean of zero
data['intercept'] = 1

train_cols = data[['intercept','Number_of_Vehicles','Number_of_Casualties','Day_of_Week','Light_Conditions',
                   'Weather_Conditions', 'Urban_or_Rural_Area',
                   'Speed_limit_20' ,'Speed_limit_30','Speed_limit_40', 'Speed_limit_50', 
                   'Speed_limit_60', 'Speed_limit_70' ]]
#train_cols.head()

test_cols = data['ACC_FATAL']
#test_cols.head()


result = sm.Logit(test_cols, train_cols).fit()
print result.summary()

# Add prediction to dataframe
data['pred'] = result.predict(train_cols)

fpr, tpr, thresholds =roc_curve(data['ACC_FATAL'], data['pred'])
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)



Optimization terminated successfully.
         Current function value: 0.064556
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:              ACC_FATAL   No. Observations:              1640597
Model:                          Logit   Df Residuals:                  1640584
Method:                           MLE   Df Model:                           12
Date:                Sun, 11 Dec 2016   Pseudo R-squ.:                 0.07134
Time:                        18:02:56   Log-Likelihood:            -1.0591e+05
converged:                       True   LL-Null:                   -1.1405e+05
                                        LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
intercept               -3.1182      0.612     -5.094      0.000        -4.318    -1.918

## Perform OLS without Categorical Variables

In [8]:
mod = smf.ols(formula='ACC_FATAL ~ Number_of_Vehicles + Number_of_Casualties + Day_of_Week + Light_Conditions + Weather_Conditions + Urban_or_Rural_Area + Speed_limit', data=df)
res = mod.fit()

In [9]:
print res.summary()

                            OLS Regression Results                            
Dep. Variable:              ACC_FATAL   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3134.
Date:                Sun, 11 Dec 2016   Prob (F-statistic):               0.00
Time:                        17:46:47   Log-Likelihood:             1.2541e+06
No. Observations:             1640597   AIC:                        -2.508e+06
Df Residuals:                 1640589   BIC:                        -2.508e+06
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept               -0.0197 

## Perform OLS Categorical Variables

In [13]:
mod = smf.ols(formula='ACC_FATAL ~ Number_of_Vehicles + Number_of_Casualties + Day_of_Week + C(Light_Conditions) + C(Weather_Conditions) + C(Urban_or_Rural_Area) + Speed_limit', data=df)
res = mod.fit()

In [12]:
print res.summary()

                            OLS Regression Results                            
Dep. Variable:              ACC_FATAL   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     957.2
Date:                Sun, 11 Dec 2016   Prob (F-statistic):               0.00
Time:                        17:49:31   Log-Likelihood:             1.2550e+06
No. Observations:             1640597   AIC:                        -2.510e+06
Df Residuals:                 1640571   BIC:                        -2.510e+06
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------------
Intercept         